In [1]:
import pandas as pd
import sqlite3
from connector import connect_to
from sqlalchemy import create_engine, text

**Блок1:** Создание схемы и таблиц

1.В базе данных создайте новую схему adv_works

In [ ]:
schema = '''
create schema if not exists adv_works;
set search_path to adv_works;'''

2.На основе данных из файла создайте в схеме adv_works таблицы и соедините их между собою по основным и внешним ключам.

In [ ]:
customer = '''
create table if not exists customers (
    customer_key int primary key,
    geography_key int,
    name varchar(100),
    birth_date date,
    marital_status char(1),
    gender char(1),
    yearly_income decimal(10, 2),
    number_children_at_home int,
    occupation varchar(50),
    house_owner_flag bit,
    number_cars_owned int,
    address_line1 varchar(100),
    address_line2 varchar(100),
    phone varchar(20),
    date_first_purchase date
);'''

territory = '''
create table if not exists territory (
    territory_key int primary key,
    region varchar(50),
    country varchar(50),
    group_name varchar(50)
);'''

product_category = '''
create table if not exists product_category (
    product_category_key int primary key,
    product_category_alternate_key int,
    english_product_category_name varchar(50),
    spanish_product_category_name varchar(50),
    french_product_category_name varchar(50)
);'''

product_subcategory = '''
create table if not exists product_subcategory (
    product_subcategory_key int primary key,
    product_subcategory_alternate_key int,
    english_product_subcategory_name varchar(50),
    spanish_product_subcategory_name varchar(50),
    french_product_subcategory_name varchar(50),
    product_category_key int
);'''

product = '''
create table if not exists products (
    product_key int primary key,
    product_subcategory_key int,
    product_name varchar(100),
    standard_cost decimal(18, 4),
    color varchar(50),
    safety_stock_level int,
    list_price decimal(18, 4),
    size varchar(20),
    size_range varchar(20),
    weight decimal(10, 2),
    days_to_manufacture int,
    product_line char(2),
    dealer_price decimal(18, 4),
    class char(1),
    model_name varchar(50),
    description text,
    start_date date,
    end_date date,
    status varchar(20)
);'''

sale = '''
create table if not exists sales (
    product_key int,
    order_date date,
    order_date_key int,
    customer_key int,
    sales_territory_key int,
    sales_order_number varchar(20),
    sales_order_line_number int,
    order_quantity int,
    unit_price decimal(18, 4),
    extended_amount decimal(18, 4),
    unit_price_discount_pct decimal(5, 2),
    discount_amount decimal(18, 4),
    product_standard_cost decimal(18, 4),
    total_product_cost decimal(18, 4),
    sales_amount decimal(18, 4),
    tax_amt decimal(18, 4),
    freight decimal(18, 4),
    region_month_id varchar(50)
);
commit;'''

foreign_keys = '''
alter table product_subcategory
    add constraint fk_product_category
    foreign key (product_category_key) references product_category(product_category_key);

alter table products
    add constraint fk_product_subcategory
    foreign key (product_subcategory_key) references product_subcategory(product_subcategory_key);

alter table sales
    add constraint fk_sales_product
    foreign key (product_key) references products(product_key);

alter table sales
    add constraint fk_sales_customer
    foreign key (customer_key) references customers(customer_key);

alter table sales
    add constraint fk_sales_territory
    foreign key (sales_territory_key) references territory(territory_key);
commit;'''

In [ ]:
with connect_to('postgres') as conn:
    conn.execute(text(schema))
    conn.execute(text(customer))
    conn.execute(text(territory))
    conn.execute(text(product_category))
    conn.execute(text(product_subcategory))
    conn.execute(text(product))
    conn.execute(text(sale))
    conn.commit()
    conn.execute(text(foreign_keys))
    conn.commit()

In [ ]:
file_path = 'adventure_works.xlsx'
excel_data = pd.ExcelFile(file_path)

def preprocess_customers(df):
    df = df.rename(columns={
        'CustomerKey': 'customer_key',
        'GeographyKey': 'geography_key',
        'Name': 'name',
        'BirthDate': 'birth_date',
        'MaritalStatus': 'marital_status',
        'Gender': 'gender',
        'YearlyIncome': 'yearly_income',
        'NumberChildrenAtHome': 'number_children_at_home',
        'Occupation': 'occupation',
        'HouseOwnerFlag': 'house_owner_flag',
        'NumberCarsOwned': 'number_cars_owned',
        'AddressLine1': 'address_line1',
        'AddressLine2': 'address_line2',
        'Phone': 'phone',
        'DateFirstPurchase': 'date_first_purchase'
    })
    df['house_owner_flag'] = df['house_owner_flag'].apply(lambda x: '1' if x else '0')
    return df

def preprocess_territory(df):
    df = df.rename(columns={
        'Territory Key': 'territory_key',
        'Region': 'region',
        'Country': 'country',
        'Group': 'group_name'
    })
    return df

def preprocess_product_category(df):
    df = df.rename(columns={
        'ProductCategoryKey': 'product_category_key',
        'ProductCategoryAlternateKey': 'product_category_alternate_key',
        'EnglishProductCategoryName': 'english_product_category_name',
        'SpanishProductCategoryName': 'spanish_product_category_name',
        'FrenchProductCategoryName': 'french_product_category_name'
    })
    return df

def preprocess_product_subcategory(df):
    df = df.rename(columns={
        'ProductSubcategoryKey': 'product_subcategory_key',
        'ProductSubcategoryAlternateKey': 'product_subcategory_alternate_key',
        'EnglishProductSubcategoryName': 'english_product_subcategory_name',
        'SpanishProductSubcategoryName': 'spanish_product_subcategory_name',
        'FrenchProductSubcategoryName': 'french_product_subcategory_name',
        'ProductCategoryKey': 'product_category_key'
    })
    return df

def preprocess_products(df):
    df = df.rename(columns={
        'ProductKey': 'product_key',
        'ProductSubcategoryKey': 'product_subcategory_key',
        'ProductName': 'product_name',
        'StandardCost': 'standard_cost',
        'Color': 'color',
        'SafetyStockLevel': 'safety_stock_level',
        'ListPrice': 'list_price',
        'Size': 'size',
        'SizeRange': 'size_range',
        'Weight': 'weight',
        'DaysToManufacture': 'days_to_manufacture',
        'ProductLine': 'product_line',
        'DealerPrice': 'dealer_price',
        'Class': 'class',
        'ModelName': 'model_name',
        'Description': 'description',
        'StartDate': 'start_date',
        'EndDate': 'end_date',
        'Status': 'status'
    })
    return df

def preprocess_sales(df):
    df = df.rename(columns={
        'ProductKey': 'product_key',
        'OrderDate': 'order_date',
        'OrderDateKey': 'order_date_key',
        'CustomerKey': 'customer_key',
        'SalesTerritoryKey': 'sales_territory_key',
        'SalesOrderNumber': 'sales_order_number',
        'SalesOrderLineNumber': 'sales_order_line_number',
        'OrderQuantity': 'order_quantity',
        'UnitPrice': 'unit_price',
        'ExtendedAmount': 'extended_amount',
        'UnitPriceDiscountPct': 'unit_price_discount_pct',
        'DiscountAmount': 'discount_amount',
        'ProductStandardCost': 'product_standard_cost',
        'TotalProductCost': 'total_product_cost',
        'SalesAmount': 'sales_amount',
        'TaxAmt': 'tax_amt',
        'Freight': 'freight',
        'RegionMonthID': 'region_month_id'
    })
    return df

customers = preprocess_customers(excel_data.parse('Customers'))
territory = preprocess_territory(excel_data.parse('Territory'))
product_category = preprocess_product_category(excel_data.parse('ProductCategory'))
product_subcategory = preprocess_product_subcategory(excel_data.parse('ProductSubCategory'))
products = preprocess_products(excel_data.parse('Products'))
sales = preprocess_sales(excel_data.parse('Sales'))


In [ ]:
with connect_to('postgres') as pg:
    pg.execute(text("set search_path to joins, adv_works"))
    pg.commit()

    customers.to_sql(
        name='customers',
        con=pg,
        index=False,
        if_exists='append'
    )
    territory.to_sql(
        name='territory',
        con=pg,
        index=False,
        if_exists='append'
    )
    product_category.to_sql(
        name='product_category',
        con=pg,
        index=False,
        if_exists='append'
    )
    product_subcategory.to_sql(
        name='product_subcategory',
        con=pg,
        index=False,
        if_exists='append'
    )
    products.to_sql(
        name='products',
        con=pg,
        index=False,
        if_exists='append'
    )
    sales.to_sql(
        name='sales',
        con=pg,
        index=False,
        if_exists='append'
    )
    
    pg.commit()

### **Блок 2.** Аналитические задачи
**Секция 1. Анализ клиентов**

1.Сегментация по доходу: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.

In [4]:
query = '''
set search_path to adv_works;
select 
    occupation,
    round(avg(yearly_income)) as avg_income, 
    count(*) as number_of_customers
from customers
group by occupation
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,occupation,avg_income,number_of_customers
0,Management,92325.0,3075
1,Clerical,30710.0,2928
2,Manual,16451.0,2384
3,Skilled Manual,51715.0,4577
4,Professional,74185.0,5520


2.Семейный профиль: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: has_children (где 1 означает - имеет детей и 0 - не имеет детей), pct_of_customer_base.

In [7]:
query = '''
set search_path to adv_works;
select 
    case when number_children_at_home > 0 then 1
    else 0 
    end as has_children,
    round(count(customer_key)*100/(select count(*) from customers)) as pct_of_customer_base
from customers
group by has_children;
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,has_children,pct_of_customer_base
0,0,60.0
1,1,39.0


3.  **Высокодоходные клиенты**: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле `SalesAmount`). Итоговая таблица должна содержать следующие поля: *customer_key*, *customer_name*, *total_purchase*.

In [9]:
query = '''
set search_path to adv_works;
select 
    s.customer_key, 
    c.name AS customer_name, 
    sum(s.sales_amount) as total_purchase
from sales s
left join customers c ON s.customer_key = c.customer_key
group by s.customer_key, c.name
order by total_purchase DESC limit 10;
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,customer_key,customer_name,total_purchase
0,12301,Nichole Nara,13295.3800
1,12132,Kaitlyn Henderson,13294.2700
2,12308,Margaret He,13269.2700
3,12131,Randall Dominguez,13265.9900
4,12300,Adriana Gonzalez,13242.7000
5,12321,Rosa Hu,13215.6500
6,12124,Brandi Gill,13195.6400
7,12307,Brad She,13173.1900
8,12296,Francisco Sara,13164.6400
9,11433,Maurice Shan,12909.6682


4.  **Влияние семейного положения**: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (*MaritalStatus*) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: *year*, *marital_status*, *avg_sales_amount*.  


In [12]:
query = '''
set search_path to adv_works;
select 
    EXTRACT(YEAR FROM s.order_date) AS year,
    c.marital_status,
    round(avg(s.sales_amount), 2) as avg_sales_amount
from sales s
left join customers c on s.customer_key = c.customer_key
group by year, c.marital_status
order by year, marital_status;
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,year,marital_status,avg_sales_amount
0,2001.0,M,3245.03
1,2001.0,S,3203.84
2,2002.0,M,2397.07
3,2002.0,S,2482.13
4,2003.0,M,378.56
5,2003.0,S,427.78
6,2004.0,M,290.64
7,2004.0,S,318.05


### Секция 2. Анализ продаж  
 1. **Ежемесячные продажи**: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *sales_count* (количество продаж), *sales_amount*.  
 

In [14]:
query = '''
set search_path to adv_works;
select 
    EXTRACT(YEAR FROM order_date) as year,
    to_char(order_date, 'YYYYMM') as monthkey,
    to_char(order_date, 'Month') as month_name,
    count(sales_order_number) as sales_count, 
    round(sum(sales_amount), 2) as sales_amount
from sales 
where EXTRACT(YEAR FROM order_date) IN (2003, 2004)
group by year, monthkey, month_name
order by year, monthkey
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,year,monthkey,month_name,sales_count,sales_amount
0,2003.0,200301,January,244,438865.17
1,2003.0,200302,February,272,489090.34
2,2003.0,200303,March,272,485574.79
3,2003.0,200304,April,294,506399.27
4,2003.0,200305,May,335,562772.56
5,2003.0,200306,June,321,554799.23
6,2003.0,200307,July,1411,886668.84
7,2003.0,200308,August,3819,847413.51
8,2003.0,200309,September,3885,1010258.13
9,2003.0,200310,October,4146,1080449.58


2. **Продажи по регионам**: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: *region*, *sales_count*, *sales_amount*.  
 

In [17]:
query = '''
set search_path to adv_works;
select 
    t.region, 
    count(s.sales_order_number) as sales_count,
    round(sum(s.sales_amount)) as sales_amount
from sales s
left join territory t on s.sales_territory_key = t.territory_key
group by t.region
order by sales_amount DESC;
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,region,sales_count,sales_amount
0,Australia,13345,9061001.0
1,Southwest,12265,5718151.0
2,Northwest,8993,3649867.0
3,United Kingdom,6906,3391712.0
4,Germany,5625,2894312.0
5,France,5558,2644018.0
6,Canada,7620,1977845.0
7,Southeast,39,12239.0
8,Northeast,27,6532.0
9,Central,20,3001.0


### Секция 3. Анализ продуктов  
 1. **Доля продаж**: Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: *year*, *product_key*, *product_category_key*, *english_product_category_name*, *sales_amount*, *pct_of_total_sales*.
 

In [21]:
query = '''
set search_path to adv_works;
with total_sales as (
    select sum(s.sales_amount) as total_sales_amount
    from sales s
)
select 
    extract(year from s.order_date) as year,
    p.product_key,
    ps.product_category_key,
    pc.english_product_category_name,
    sum(s.sales_amount) as sales_amount,
    round(sum(s.sales_amount) * 100.0 / (select total_sales_amount from total_sales), 2) as pct_of_total_sales
from sales s
left join products p on s.product_key = p.product_key
left join product_subcategory ps on p.product_subcategory_key = ps.product_subcategory_key
left join product_category pc on ps.product_category_key = pc.product_category_key
group by year, p.product_key, ps.product_category_key, pc.english_product_category_name
order by year, pct_of_total_sales desc;
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,year,product_key,product_category_key,english_product_category_name,sales_amount,pct_of_total_sales
0,2001.0,310,1,Bikes,593992.82,2.02
1,2001.0,312,1,Bikes,547475.31,1.86
2,2001.0,311,1,Bikes,500957.80,1.71
3,2001.0,314,1,Bikes,486644.72,1.66
4,2001.0,313,1,Bikes,472331.64,1.61
...,...,...,...,...,...,...
311,2004.0,463,3,Clothing,6685.77,0.02
312,2004.0,484,4,Accessories,4173.75,0.01
313,2004.0,481,3,Clothing,1474.36,0.01
314,2004.0,480,4,Accessories,4202.15,0.01


2. **Самые продаваемые продукты**: Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: *product_key*, *product_name*, *english_product_category_name*, *sales_amount*
 

In [16]:
query = '''
set search_path to adv_works;
SELECT 
    p.product_key,
    p.product_name,
    pc.english_product_category_name,
    SUM(s.sales_amount) AS sales_amount
FROM sales s
left join products p on s.product_key = p.product_key
left join product_subcategory ps on p.product_subcategory_key = ps.product_subcategory_key
left join product_category pc on ps.product_category_key = pc.product_category_key
GROUP BY 
    p.product_key, p.product_name, pc.english_product_category_name
ORDER BY 
    sales_amount DESC
LIMIT 5;

'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,product_key,product_name,english_product_category_name,sales_amount
0,312,"Road-150 Red, 48",Bikes,1205876.99
1,310,"Road-150 Red, 62",Bikes,1202298.72
2,313,"Road-150 Red, 52",Bikes,1080637.54
3,314,"Road-150 Red, 56",Bikes,1055589.65
4,311,"Road-150 Red, 44",Bikes,1005493.87


3. **Маржа от продаж**: Посчитайте разницу между суммой продаж (*SalesAmount*) за минусом себестоимости (*TotalProductCost*), налогов (*TaxAmt) и расходов на доставку (*Freight*) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *product_key*, *product_name*, *sales_amount*, *total_product_cost*, *tax_amt*, *freight*, *margin*, *margin_pct* (маржа как процент от суммы продаж).


In [25]:
query = '''
set search_path to adv_works;
select 
    extract(year from s.order_date) as year,
    to_char(s.order_date, 'YYYYMM') as monthkey,
    to_char(s.order_date, 'Month') as month_name,
    p.product_key,
    p.product_name,
    sum(s.sales_amount) as sales_amount,
    sum(s.total_product_cost) as total_product_cost,
    sum(s.tax_amt) as tax_amt,
    sum(s.freight) as freight,
    round(sum(s.sales_amount) - sum(s.total_product_cost) - sum(s.tax_amt) - sum(s.freight), 2) as margin,
    round((sum(s.sales_amount) - sum(s.total_product_cost) - sum(s.tax_amt) - sum(s.freight))*100/ sum(s.sales_amount), 2) as margin_pct
from sales s
left join products p on s.product_key = p.product_key
group by year, monthkey, month_name, p.product_key, p.product_name
order by year, monthkey, margin desc
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,year,monthkey,month_name,product_key,product_name,sales_amount,total_product_cost,tax_amt,freight,margin,margin_pct
0,2001.0,200107,July,312,"Road-150 Red, 48",100191.56,60796.2376,8015.3248,2504.7904,28875.21,28.82
1,2001.0,200107,July,311,"Road-150 Red, 44",82300.21,49939.7666,6584.0168,2057.5064,23718.92,28.82
2,2001.0,200107,July,310,"Road-150 Red, 62",78721.94,47768.4724,6297.7552,1968.0496,22687.66,28.82
3,2001.0,200107,July,314,"Road-150 Red, 56",53674.05,32569.4130,4293.9240,1341.8520,15468.86,28.82
4,2001.0,200107,July,313,"Road-150 Red, 52",42939.24,26055.5304,3435.1392,1073.4816,12375.09,28.82
...,...,...,...,...,...,...,...,...,...,...,...
1895,2004.0,200407,July,481,"Racing Socks, M",152.83,57.1591,12.2264,3.8216,79.62,52.10
1896,2004.0,200407,July,488,"Short-Sleeve Classic Jersey, S",593.89,457.2953,47.5112,14.8478,74.24,12.50
1897,2004.0,200407,July,490,"Short-Sleeve Classic Jersey, L",431.92,332.5784,34.5536,10.7984,53.99,12.50
1898,2004.0,200407,July,479,Road Bottle Cage,89.90,33.6230,7.1920,2.2480,46.84,52.10


### Секция 4. Анализ трендов  
1. **Квартальный рост**: Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: *year*, *quarter_id*, *product_category_key*, *english_product_category_name*, *quarter_sales_amount*, *quarter_over_quarter_growth_pct*.


2. **Сравнение будних и выходных (суббота, воскресенье) дней**: Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: *year*, *day_name*, *is_weekend* (где 1 означает выходной а 0 будний день), *sales_amount*.


In [9]:
query = '''
set search_path to adv_works;
select 
    EXTRACT(YEAR FROM s.order_date) AS year,
    to_char(s.order_date, 'Day') as day_name,
    case 
        when to_char(s.order_date, 'Day') in ('Saturday ', 'Sunday   ')  then 1 else 0
        end as is_weekend,
    round(sum(s.sales_amount), 2) AS sales_amount
from sales s
group by year, day_name, is_weekend
order by year, is_weekend DESC, sales_amount DESC
'''
with connect_to('postgres') as pg:
    df = pd.read_sql(query, pg)

df

,year,day_name,is_weekend,sales_amount
0,2001.0,Sunday,1,526026.82
1,2001.0,Saturday,1,505234.58
2,2001.0,Friday,0,468724.50
3,2001.0,Thursday,0,450281.09
4,2001.0,Monday,0,447197.10
5,2001.0,Wednesday,0,435300.15
6,2001.0,Tuesday,0,433609.42
7,2002.0,Sunday,1,937525.27
8,2002.0,Saturday,1,915346.19
9,2002.0,Wednesday,0,1004132.60
